# LENET ARCHITECTURE

In [1]:
import tensorflow
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation

2024-10-29 10:18:23.943838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730177303.956197   17562 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730177303.959664   17562 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 10:18:23.972996: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


In [7]:
# Create an ImageDataGenerator for data augmentation and preprocessing

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

In [9]:
# Define the path to your dataset directory
dataset_dir = 'Dataset'

# Set the batch size and number of classes
batch_size = 64
num_classes = 4

In [11]:
# Load and preprocess the dataset using the ImageDataGenerator

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

Found 640 images belonging to 4 classes.


In [13]:
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')


Found 160 images belonging to 4 classes.


In [15]:
from tensorflow.keras.models import Sequential

MODEL=Sequential()
MODEL.add(Convolution2D(filters=32, kernel_size=(3,3), strides=(3,3), input_shape=(224,224,3), padding=('valid'), activation='relu'))
MODEL.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
MODEL.add(Convolution2D(filters=128, kernel_size=(3,3), strides=(3,3), padding=('valid'), activation='relu'))
MODEL.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
MODEL.add(Flatten())
MODEL.add(Dense(256, activation='relu'))
MODEL.add(Dense(4, activation='softmax'))

OPT    = tensorflow.keras.optimizers.Adam(0.001)

MODEL.compile(optimizer=OPT,loss='categorical_crossentropy',metrics=["accuracy", tensorflow.keras.metrics.Precision()])
MODEL.summary()

/home/sandy3o/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-29 10:18:34.648236: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 74, 74, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 12, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,218,820 (4.65 MB)

 Trainable params: 1,218,820 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model_path = "LENET.h5"

from keras.callbacks import ModelCheckpoint
M = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True)

ModuleNotFoundError: No module named 'callbacks'

In [19]:
epochs = 100

In [21]:
history = MODEL.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[M]  # Add the callbacks here
)

NameError: name 'M' is not defined

Epoch 72/100
10/10 [==============================] - ETA: 0s - loss: 0.3448 - accuracy: 0.8687 - precision: 0.8770
Epoch 72: val_accuracy did not improve from 0.90625
10/10 [==============================] - 10s 1s/step - loss: 0.3448 - accuracy: 0.8687 - precision: 0.8770 - val_loss: 0.3399 - val_accuracy: 0.8562 - val_precision: 0.8608
Epoch 73/100
10/10 [==============================] - ETA: 0s - loss: 0.3976 - accuracy: 0.8234 - precision: 0.8363
Epoch 73: val_accuracy did not improve from 0.90625
10/10 [==============================] - 9s 970ms/step - loss: 0.3976 - accuracy: 0.8234 - precision: 0.8363 - val_loss: 0.5884 - val_accuracy: 0.7812 - val_precision: 0.7922
Epoch 74/100
10/10 [==============================] - ETA: 0s - loss: 0.4461 - accuracy: 0.8109 - precision: 0.8252
Epoch 74: val_accuracy improved from 0.90625 to 0.91875, saving model to LENET.h5
10/10 [==============================] - 10s 1000ms/step - loss: 0.4461 - accuracy: 0.8109 - precision: 0.8252 - val_l

Epoch 96/100
10/10 [==============================] - ETA: 0s - loss: 0.2982 - accuracy: 0.8844 - precision: 0.8947
Epoch 96: val_accuracy did not improve from 0.92500
10/10 [==============================] - 10s 968ms/step - loss: 0.2982 - accuracy: 0.8844 - precision: 0.8947 - val_loss: 0.2862 - val_accuracy: 0.9187 - val_precision: 0.9245
Epoch 97/100
10/10 [==============================] - ETA: 0s - loss: 0.3189 - accuracy: 0.8766 - precision: 0.8876
Epoch 97: val_accuracy did not improve from 0.92500
10/10 [==============================] - 9s 954ms/step - loss: 0.3189 - accuracy: 0.8766 - precision: 0.8876 - val_loss: 0.3111 - val_accuracy: 0.9062 - val_precision: 0.9161
Epoch 98/100
10/10 [==============================] - ETA: 0s - loss: 0.2950 - accuracy: 0.8891 - precision: 0.8965
Epoch 98: val_accuracy did not improve from 0.92500
10/10 [==============================] - 10s 955ms/step - loss: 0.2950 - accuracy: 0.8891 - precision: 0.8965 - val_loss: 0.3856 - val_accuracy: 

In [23]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20, 8))
plt.plot(history.history['accuracy'])

for i in range(34):
    if i%5 == 0:
        plt.annotate(np.round(history.history['accuracy'][i]*100,2),xy=(i,history.history['accuracy'][i]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

NameError: name 'history' is not defined

<Figure size 2000x800 with 0 Axes>

In [25]:
plt.figure(figsize=(20, 8))
plt.plot(history.history['loss'])

for i in range(34):
    if i%5 == 0:
        plt.annotate(np.round(history.history['loss'][i]*100,2),xy=(i,history.history['loss'][i]))

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

NameError: name 'history' is not defined

<Figure size 2000x800 with 0 Axes>